In [213]:
import regex as re
from dataclasses import dataclass, field

In [214]:
paterns = {
    r">\s*\[!SUMMARY\]" : "tableofcontents",
    r">\s*\[!question\]" : "question",
    r">\s*\[!definition\]" : "definition",
    r">\s*\[!propriete\]" : "propriete",
    r">\s*\[!attention\]" : "attention"
}

In [215]:
def compte_tab(line) : 
    """Compte le nombre de " > " avant le premier caractère non espace

    Args:
        line (str): une ligne de texte Md
    """
    count = 0
    for char in line :
        if char == '>':
            count += 1
        elif char.isspace():
            continue
        else:
            break
    return count

In [216]:
class bloc_environement:
    def __init__(self, type_bloc : str, niveau : int, i_debut, i_fin):
        self.type_bloc = type_bloc
        self.niveau = niveau
        self.i_debut = i_debut
        self.i_fin = i_fin
        self.content = []
        
    def __str__(self):
        return f"Bloc de type {self.type_bloc} : niveau : {self.niveau} ; Intervalle : [{self.i_debut}, {self.i_fin}]"
    
    def __repr__(self):
        return f"Bloc de type {self.type_bloc} : niveau : {self.niveau} ; Intervalle : [{self.i_debut}, {self.i_fin}]"


In [220]:
def calcul_bloc(text : list[str], niveau : int) -> list[bloc_environement]:
    """Calcul recursivement les blocs imbriqués

    Args:
        text (list[str]): liste de lignes de texte Md
        niveau (int): niveau d'imbrication du bloc
    """
    bloc_liste = []
    if text != []:
        in_bloc = False
        
        for ligne in text:
            if in_bloc:
                if compte_tab(ligne) == niveau:
                    i_fin = text.index(ligne) - 1
                    in_bloc = False
                    bloc = bloc_environement(type_bloc, niveau + 1, i_debut, i_fin)
                    bloc.content = text[i_debut + 1 : i_fin + 1]
                    bloc_liste.append(bloc)
                    if i_debut + 1 > i_fin: # pas besoin de faire les calculs s'il l'on a la fin = debut
                        bloc_liste_recursif = calcul_bloc(text[i_debut + 1 : i_fin + 1], niveau + 1)
                    try : 
                        bloc_liste += bloc_liste_recursif
                    except TypeError :
                           print("Erreur lors de l'ajout du bloc : ", bloc_liste , bloc_liste_recursif, type(bloc_liste_recursif))                         
                        
            else:
                if compte_tab(ligne) > niveau:
                    i_debut = text.index(ligne)
                    type_bloc = None
                    while type_bloc is None:
                        for patern in paterns:
                            if re.search(patern, ligne):
                                type_bloc = paterns[patern]
                                break
                        type_bloc = "NoType"
                    in_bloc = True
    return bloc_liste
                    

In [ ]:
def main():
    fichier = "Fichier test/test.md"
    text = open(fichier, 'r', encoding='utf-8').read()
    text = text.split('\n')
    pile = []
    taille_pile = 0
    res = calcul_bloc(text, 0)
    print(calcul_bloc(text, 0))
    for bloc in res:
        print(bloc.content)
    
                    

In [222]:
main()

UnboundLocalError: cannot access local variable 'bloc_liste_recursif' where it is not associated with a value